In [1]:
# https://taku910.github.io/mecab/
import MeCab

# https://taku910.github.io/cabocha/
import CaboCha

import re

In [2]:
text = """
メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。
"""

In [3]:
print(text)


メロスは激怒した。
必ず、かの邪智暴虐の王を除かなければならぬと決意した。
メロスには政治がわからぬ。
メロスは、村の牧人である。
笛を吹き、羊と遊んで暮して来た。
けれども邪悪に対しては、人一倍に敏感であった。



In [4]:
tagger = MeCab.Tagger()

def get_parse_list(text: str) -> list[list[str]]:
    return [
        line.split("\t")
        for line in tagger.parse(text).splitlines()
        if line and line != "EOS"
    ]

In [5]:
# 30
verbs = []

for l in get_parse_list(text):
    surface = l[0]
    feature = l[4]    
    if feature.startswith("動詞"):
        verbs.append(surface)
print(verbs)

['し', '除か', 'なら', 'し', 'わから', 'ある', '吹き', '遊ん', '暮し', '来', '対し', 'あっ']


In [6]:
# 31
base_verbs = []

for l in get_parse_list(text):
    base = l[3]
    feature = l[4]    
    if feature.startswith("動詞"):
        base_verbs.append(base)
print(base_verbs)

['為る', '除く', '成る', '為る', '分かる', '有る', '吹く', '遊ぶ', '暮らす', '来る', '対する', '有る']


In [7]:
# 32
tokens = []
for l in get_parse_list(text):
    surface = l[0]
    feature = l[4]
    features = feature.split("-")
    tokens.append((surface, features[0]))
    
results = []
for i in range(len(tokens) - 2):
    if tokens[i][1] == "名詞" and tokens[i + 1][0] == "の" and tokens[i + 2][1] == "名詞":
        results.append(tokens[i][0] + "の" + tokens[i + 2][0])

print(results)

['暴虐の王', '村の牧人']


In [8]:
cabocha = CaboCha.Parser()
tree = cabocha.parse(text)

In [9]:
# 33

from_to = {}
ch_map = {}
ch = ""
ch_no = ""
symbol = False


lines = tree.toString(CaboCha.CABOCHA_FORMAT_LATTICE).splitlines()
for l in lines:
    if l == "EOS":
        break
    if l.startswith("*"):
        if symbol:
            from_to.pop(ch_no)
            symbol = False
        ch_map[ch_no] = ch
        ch = ""
        s = l.split("\u0020")
        ch_no = s[1]
        if not s[2].startswith("-"):
            from_to[ch_no] = re.findall(r"\d+", s[2])[0]
    else:
        sp = l.split("\t")
        ch += sp[0]
        if sp[1].startswith("記号"):
            symbol = True
ch_map[ch_no] = ch


for i in from_to.keys():
    print(f"{ch_map[i]}\t{ch_map[from_to[i]]}")

メロスは	激怒した。
かの	邪智暴虐の
邪智暴虐の	王を
王を	除かなければならぬと
除かなければならぬと	決意した。
メロスには	わからぬ。
政治が	わからぬ。
村の	牧人である。
笛を	吹き、
羊と	遊んで
遊んで	暮して来た。
けれども	敏感であった。
人一倍に	敏感であった。


In [10]:
# 34

ch = ""
memo_ch_no = -1
target_ch_no = -1
target_flag = False
verbs = []


lines = tree.toString(CaboCha.CABOCHA_FORMAT_LATTICE).splitlines()
for l in lines:
    if l == "EOS":
        break
    if l.startswith("*"):
        target_flag = False
        if 0 < len(ch):
            verbs.append(ch)
            ch = ""
        s = l.split("\u0020")
        if not s[2].startswith("-"):
            memo_ch_no = re.findall(r"\d+", s[2])[0]
        if s[1] == target_ch_no:
            target_ch_no = memo_ch_no
            target_flag = True
    else:
        sp = l.split("\t")
        if sp[0] == "メロス":
            target_ch_no = memo_ch_no
        if target_flag:
            ch += sp[0]
if 0 < len(ch):
    verbs.append(ch)

    
print(verbs)

['激怒した。', '決意した。', 'わからぬ。', '牧人である。', '暮して来た。', '敏感であった。']


In [11]:
# 35

print(cabocha.parseToString(text))

            メロスは-D                                          
            激怒した。-----------D                              
                  必ず、-------D |                              
                      かの-D   | |                              
                  邪智暴虐の-D | |                              
                          王を-D |                              
            除かなければならぬと-D                              
                        決意した。-----D                        
                          メロスには---D                        
                                政治が-D                        
                              わからぬ。-----D                  
                                メロスは、---D                  
                                        村の-D                  
                                  牧人である。---------D        
                                            笛を-D     |        
                                            吹き、-----D        
                           